# Modern NLP in Python - News Analysis

## Content
* [General & Background](#General-&-Background)
* [Data](#Data)
* [spaCy - Industrial-Strength NLP in Python](#spaCy---Industrial-Strength-NLP-in-Python)
* [Phrase Modeling](#Phrase-Modeling)
* [Topic Modeling with Latent Dirichlet Allocation (_LDA_)](#Topic-Modeling-with-Latent-Dirichlet-Allocation)
* [Word Vector Embedding with Word2Vec](#Word-Vector-Embedding-with-Word2Vec)

## General & Background
#### Before we get started...
__whois?__
- Patrick Harrison
- Lead Data Scientist @ S&P Global Market Intelligence - _**we are hiring**_
- University of Virginia &mdash; Systems Engineering
- patrick@skipgram.io / @skipgram

__Join Charlottesville Data Science!__
- On Meetup.com ... http://www.meetup.com/CharlottesvilleDataScience
- On Slack ... __https://cville.typeform.com/to/UEzMVh__
  - _link invites you to join the Cville team on Slack. Join Cville, then join channel __#datascience__._
  
_Note: I presented this notebook as a tutorial during the [PyData DC 2016 conference](http://pydata.org/dc2016/schedule/presentation/11/). To view the video of the presentation on YouTube, see [here](https://www.youtube.com/watch?v=6zm9NC9uRkk)._

### Our Trail Map
This tutorial features an end-to-end data science & natural language processing pipeline, starting with **raw data** and running through **preparing**, **modeling**, **visualizing**, and **analyzing** the data. We'll touch on the following points:
1. A tour of the dataset
1. Introduction to text processing with spaCy
1. Automatic phrase modeling
1. Topic modeling with LDA
1. Visualizing topic models with pyLDAvis
1. Word vector models with word2vec
1. Visualizing word2vec with t-SNE

...and we might even learn a thing or two about Python along the way.

Let's get started!

In [2]:
import pandas as pd
import os
import codecs

intermediate_directory = os.path.join('.', 'intermediate') # Directory for storing data

## Data

This section is for loading and processing raw data. Data could be stored in different stuctures or in difference file formats. The output from this section should be a list of corpus, to be used for further NLP processing and analysis.

In [3]:
df = pd.read_pickle('df_save.pck')
df = df[df['Language'].isin(['en'])]
# Combine title and contents
df['Text'] = df['Title'] + '. ' + df['Content']
df.head()

,Date,ArticleCode,Language,Title,Content,Text
0,2015-01-05,1,en,WTO celebrates 20 years of helping global econ...,"GENEVA, Jan 1 (KUNA) -- The World Trade Organi...",WTO celebrates 20 years of helping global econ...
2,2015-01-05,3,en,"Delegates of 45 nations,WTO Chief to be at CII...","New Delhi, Jan 2 (PTI) Over 1,000 delegates fr...","Delegates of 45 nations,WTO Chief to be at CII..."
3,2015-01-05,4,en,"Compliance Rulings On Aircraft, Tuna, COOL To ...","World Trade Online Posted: December 30, 2014 A...","Compliance Rulings On Aircraft, Tuna, COOL To ..."
4,2015-01-05,5,en,Sudan making new push at WTO membership: official,"Sudan Tribune 4 January 2015 January 3, 2015 (...",Sudan making new push at WTO membership: offic...
5,2015-01-05,6,en,China Ends Rare-Earth Minerals Export Quotas,China Ends Decade-Old Quota System Limiting Ex...,China Ends Rare-Earth Minerals Export Quotas. ...


Normalize words used in articles

In [5]:
# Normalize words
replacements = {
    'Text': {
        r'\'s': '', 
        'Indian': 'India', 
        'nextgeneration': 'next generation', 
        '//iconnect\.wto\.org/': '', 
        '-': ' ', 
        'U.S.': 'United States', 
        ' US ': 'United States', 
        'S.Korea': 'South Korea', 
        'S. Korea': 'South Korea', 
        'WTO': 'world trade organization', 
        '‘': '', 
        'imports': 'import', 
        'Imports': 'import', 
        'exports': 'export', 
        'Exports': 'export', 
        'NZ ': 'New Zealand ', 
        '\"': '',
        '\'': '', 
    }
}
df.replace(replacements, regex=True, inplace=True)

Convert DF to lists

In [6]:
texts = df['Text'].tolist()
titles = df['Title'].tolist()
dates = df['Date'].tolist()
articlecodes = df['ArticleCode'].tolist()
print(str(len(texts)) + ' texts')
print(str(len(titles)) + ' titles')
print(str(len(dates)) + ' dates')
print(str(len(articlecodes)) + ' articlecodes')

6409 texts
6409 titles
6409 dates
6409 articlecodes


## spaCy - Industrial-Strength NLP in Python

![spaCy](https://s3.amazonaws.com/skipgram-images/spaCy.png)

[**spaCy**](https://spacy.io) is an industrial-strength natural language processing (_NLP_) library for Python. spaCy's goal is to take recent advancements in natural language processing out of research papers and put them in the hands of users to build production software.

spaCy handles many tasks commonly associated with building an end-to-end natural language processing pipeline:
- Tokenization
- Text normalization, such as lowercasing, stemming/lemmatization
- Part-of-speech tagging
- Syntactic dependency parsing
- Sentence boundary detection
- Named entity recognition and annotation

In the "batteries included" Python tradition, spaCy contains built-in data and models which you can use out-of-the-box for processing general-purpose English language text:
- Large English vocabulary, including stopword lists
- Token "probabilities"
- Word vectors

spaCy is written in optimized Cython, which means it's _fast_. According to a few independent sources, it's the fastest syntactic parser available in any language. Key pieces of the spaCy parsing pipeline are written in pure C, enabling efficient multithreading (i.e., spaCy can release the _GIL_).

In [7]:
import spacy
# conda install -c spacy sputnik=0.9.3  >> Restart
# set https_proxy=https://wwwproxy.unicc.org:8080 >> 
# set http_proxy=http://wwwproxy.unicc.org:8080 >> python -m spacy.en.download
import itertools as it
nlp = spacy.load('en')

Let's grab a sample review to play with.

In [8]:
sample_article = texts[0]
sample_article

'world trade organization celebrates 20 years of helping global economy grow. GENEVA, Jan 1 (KUNA)    The World Trade Organization (world trade organization) celebrated Thursday the 20th anniversary of its foundation to be the only global international organization dealing with the rules of trade between nations. Over the past 20 years, this organization has, on balance, made an important contribution to the global economy and to smoother trading relations between nations, world trade organization Director General Roberto Azevedo said in a press release. 20 years ago, on 1 January 1995, the world trade organization opened its doors for business. Since then this organization, and the system of transparent, multilaterally agreed rules that it embodies, has made a major contribution to the strength and stability of the global economy, Azevedo said in a press release from the world trade organization headquarters in Geneva. He pointed that the world trade organization, over the past two de

In [9]:
%%time
parsed_article = nlp(sample_article)

Wall time: 47.2 ms


...1/20th of a second or so. Let's take a look at what we got during that time...

In [10]:
print(parsed_article) 

world trade organization celebrates 20 years of helping global economy grow. GENEVA, Jan 1 (KUNA)    The World Trade Organization (world trade organization) celebrated Thursday the 20th anniversary of its foundation to be the only global international organization dealing with the rules of trade between nations. Over the past 20 years, this organization has, on balance, made an important contribution to the global economy and to smoother trading relations between nations, world trade organization Director General Roberto Azevedo said in a press release. 20 years ago, on 1 January 1995, the world trade organization opened its doors for business. Since then this organization, and the system of transparent, multilaterally agreed rules that it embodies, has made a major contribution to the strength and stability of the global economy, Azevedo said in a press release from the world trade organization headquarters in Geneva. He pointed that the world trade organization, over the past two dec

Looks the same! What happened under the hood?

What about sentence detection and segmentation?

In [15]:
# To understand enumerate() function
# for sentence in parsed_article.sents:
#     print(sentence) 
#     print('')

In [16]:
for num, sentence in enumerate(parsed_article.sents):
    print('Sentence {}:'.format(num + 1)) 
    print(sentence) 
    print('')

Sentence 1:
world trade organization celebrates 20 years of helping global economy grow.

Sentence 2:
GENEVA, Jan 1 (KUNA)    

Sentence 3:
The World Trade Organization (world trade organization) celebrated Thursday the 20th anniversary of its foundation to be the only global international organization dealing with the rules of trade between nations.

Sentence 4:
Over the past 20 years, this organization has, on balance, made an important contribution to the global economy and to smoother trading relations between nations, world trade organization Director General Roberto Azevedo said in a press release.

Sentence 5:
20 years ago, on 1 January 1995, the world trade organization opened its doors for business.

Sentence 6:
Since then this organization, and the system of transparent, multilaterally agreed rules that it embodies, has made a major contribution to the strength and stability of the global economy, Azevedo said in a press release from the world trade organization headquarters 

What about named entity detection?

In [17]:
for num, entity in enumerate(parsed_article.ents):
    print('Entity {}:'.format(num + 1), entity, '-', entity.label_)
#     print('')

Entity 1: 20 years - DATE
Entity 2: GENEVA - GPE
Entity 3: Jan 1 - DATE
Entity 4: The World Trade Organization - ORG
Entity 5: Thursday - DATE
Entity 6: the past 20 years - DATE
Entity 7: Roberto Azevedo - PERSON
Entity 8: 20 years ago - DATE
Entity 9: 1 January 1995 - DATE
Entity 10: Azevedo - PERSON
Entity 11: Geneva - GPE
Entity 12: the past two decades - DATE
Entity 13: Azevedo - PERSON
Entity 14: 2008 - DATE
Entity 15: 1995 - DATE
Entity 16: 33 - CARDINAL
Entity 17: Today - DATE
Entity 18: 160 - CARDINAL
Entity 19: approximately 98 per cent - PERCENT
Entity 20: 9th Ministerial Conference - ORG
Entity 21: Bali - GPE
Entity 22: 2013 - DATE
Entity 23: first - ORDINAL
Entity 24: Bali - GPE
Entity 25: December 2014 - DATE
Entity 26: Bali - GPE
Entity 27: Azevedo - GPE
Entity 28: 2015 - DATE
Entity 29: Bali - GPE
Entity 30: the Doha Development Agenda - ORG


What about part of speech tagging?

In [18]:
token_text = [token.orth_ for token in parsed_article]
token_pos = [token.pos_ for token in parsed_article]
pd.DataFrame(list(zip(token_text, token_pos)),
             columns=['token_text', 'part_of_speech'])

,token_text,part_of_speech
0,world,NOUN
1,trade,NOUN
2,organization,NOUN
3,celebrates,VERB
4,20,NUM
5,years,NOUN
6,of,ADP
7,helping,VERB
8,global,ADJ
9,economy,NOUN


What about text normalization, like stemming/lemmatization and shape analysis?

In [19]:
token_lemma = [token.lemma_ for token in parsed_article]
token_shape = [token.shape_ for token in parsed_article]

pd.DataFrame(list(zip(token_text, token_lemma, token_shape)),
             columns=['token_text', 'token_lemma', 'token_shape'])

,token_text,token_lemma,token_shape
0,world,world,xxxx
1,trade,trade,xxxx
2,organization,organization,xxxx
3,celebrates,celebrate,xxxx
4,20,20,dd
5,years,year,xxxx
6,of,of,xx
7,helping,help,xxxx
8,global,global,xxxx
9,economy,economy,xxxx


What about token-level entity analysis?

In [20]:
token_entity_type = [token.ent_type_ for token in parsed_article]
token_entity_iob = [token.ent_iob_ for token in parsed_article]

pd.DataFrame(list(zip(token_text, token_entity_type, token_entity_iob)),
             columns=['token_text', 'entity_type', 'inside_outside_begin'])

,token_text,entity_type,inside_outside_begin
0,world,,O
1,trade,,O
2,organization,,O
3,celebrates,,O
4,20,DATE,B
5,years,DATE,I
6,of,,O
7,helping,,O
8,global,,O
9,economy,,O


What about a variety of other token-level attributes, such as the relative frequency of tokens, and whether or not a token matches any of these categories?
- stopword
- punctuation
- whitespace
- represents a number
- whether or not the token is included in spaCy's default vocabulary?

In [23]:
token_attributes = [(token.orth_,
                     token.lemma_,
                     token.prob,
                     token.is_stop,
                     token.is_punct,
                     token.is_space,
                     token.like_num,
                     token.is_oov)
                    for token in parsed_article]

df = pd.DataFrame(token_attributes,
                  columns=['text',
                           'lemma',
                           'log_probability',
                           'stop?',
                           'punctuation?',
                           'whitespace?',
                           'number?',
                           'out of vocab.?'])

df.loc[:, 'stop?':'out of vocab.?'] = (df.loc[:, 'stop?':'out of vocab.?']
                                       .applymap(lambda x: u'Yes' if x else u''))
                                               
df

,text,lemma,log_probability,stop?,punctuation?,whitespace?,number?,out of vocab.?
0,world,world,-7.874412,,,,,
1,trade,trade,-9.137971,,,,,
2,organization,organization,-10.665580,,,,,
3,celebrates,celebrate,-14.213387,,,,,
4,20,20,-8.755866,,,,Yes,
5,years,year,-7.368988,,,,,
6,of,of,-4.275874,Yes,,,,
7,helping,help,-10.098352,,,,,
8,global,global,-10.712088,,,,,
9,economy,economy,-10.065805,,,,,


If the text you'd like to process is general-purpose English language text (i.e., not domain-specific, like medical literature), spaCy is ready to use out-of-the-box.

I think it will eventually become a core part of the Python data science ecosystem &mdash; it will do for natural language computing what other great libraries have done for numerical computing.

## Phrase Modeling

_Phrase modeling_ is another approach to learning combinations of tokens that together represent meaningful multi-word concepts. We can develop phrase models by looping over the the words in our reviews and looking for words that _co-occur_ (i.e., appear one after another) together much more frequently than you would expect them to by random chance. The formula our phrase models will use to determine whether two tokens $A$ and $B$ constitute a phrase is:

$$\frac{count(A\ B) - count_{min}}{count(A) * count(B)} * N > threshold$$

...where:
* $count(A)$ is the number of times token $A$ appears in the corpus
* $count(B)$ is the number of times token $B$ appears in the corpus
* $count(A\ B)$ is the number of times the tokens $A\ B$ appear in the corpus *in order*
* $N$ is the total size of the corpus vocabulary
* $count_{min}$ is a user-defined parameter to ensure that accepted phrases occur a minimum number of times
* $threshold$ is a user-defined parameter to control how strong of a relationship between two tokens the model requires before accepting them as a phrase

Once our phrase model has been trained on our corpus, we can apply it to new text. When our model encounters two tokens in new text that identifies as a phrase, it will merge the two into a single new token.

Phrase modeling is superficially similar to named entity detection in that you would expect named entities to become phrases in the model (so _new york_ would become *new\_york*). But you would also expect multi-word expressions that represent common concepts, but aren't specifically named entities (such as _happy hour_) to also become phrases in the model.

We turn to the indispensible [**gensim**](https://radimrehurek.com/gensim/index.html) library to help us with phrase modeling &mdash; the [**Phrases**](https://radimrehurek.com/gensim/models/phrases.html) class in particular.

In [24]:
# suppress Gensim's warning message
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim') 

from gensim.models import Phrases
from gensim.models.word2vec import LineSentence

As we're performing phrase modeling, we'll be doing some iterative data transformation at the same time. Our roadmap for data preparation includes:

1. Segment text of complete reviews into sentences & normalize text
1. First-order phrase modeling $\rightarrow$ _apply first-order phrase model to transform sentences_
1. Second-order phrase modeling $\rightarrow$ _apply second-order phrase model to transform sentences_
1. Apply text normalization and second-order phrase model to text of complete reviews

We'll use this transformed data as the input for some higher-level modeling approaches in the following sections.

First, let's define a few helper functions that we'll use for text normalization. In particular, the `lemmatized_sentence_corpus` generator function will use spaCy to:
- Iterate over the 1M reviews in the `review_txt_all.txt` we created before
- Segment the reviews into individual sentences
- Remove punctuation and excess whitespace
- Lemmatize the text

... and do so efficiently in parallel, thanks to spaCy's `nlp.pipe()` function.

In [ ]:
review_txt_filepath = os.path.join(intermediate_directory, 'review_text_all.txt')

In [ ]:
with codecs.open(review_txt_filepath, 'w', encoding='utf_8') as review_txt_file:
    # loop through all reviews in the existing file and convert to dict
    for text in texts:
        # escape newline characters in the original review text
        review_txt_file.write(text + '\n')

In [31]:
# Customized

def punct_space(token):
    """
    helper function to eliminate tokens
    that are pure punctuation or whitespace
    """
    
    return token.is_punct or token.is_space

def line_review(filename):
    """
    generator function to read in reviews from the file
    and un-escape the original line breaks in the text
    """
    
    with codecs.open(filename, encoding='utf_8') as f:
        for review in f:
            yield review.replace('\\n', '\n')
            
def lemmatized_sentence_corpus(filename):
    """
    generator function to use spaCy to parse reviews,
    lemmatize the text, and yield sentences
    """
    for parsed_article in nlp.pipe(line_review(filename),batch_size=10000, n_threads=4):
    #for parsed_article in nlp.pipe(texts, batch_size=10000, n_threads=4):
        for sent in parsed_article.sents:
            yield u' '.join([token.lemma_ for token in sent
                             if not punct_space(token)])

In [28]:
unigram_sentences_filepath = os.path.join(intermediate_directory, 'unigram_sentences_all.txt')

Let's use the `lemmatized_sentence_corpus` generator to loop over the original review text, segmenting the reviews into individual sentences and normalizing the text. We'll write this data back out to a new file (`unigram_sentences_all`), with one normalized sentence per line. We'll use this data for learning our phrase models.

In [32]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to execute data prep yourself.
if 0 == 0:
    with codecs.open(unigram_sentences_filepath, 'w', encoding='utf_8') as f:
        for sentence in lemmatized_sentence_corpus(review_txt_filepath):
            f.write(sentence + '\n')

Wall time: 4min 36s


In [19]:
# %%time

# # this is a bit time consuming - make the if statement True
# # if you want to execute data prep yourself.
# if 0 == 0:

#     with codecs.open(unigram_sentences_filepath, 'w', encoding='utf_8') as f:
#         for sentence in lemmatized_sentence_corpus(texts):
#             f.write(sentence + '\n')

Wall time: 4min 5s


If your data is organized like our `unigram_sentences_all` file now is &mdash; a large text file with one document/sentence per line &mdash; gensim's [**LineSentence**](https://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.LineSentence) class provides a convenient iterator for working with other gensim components. It *streams* the documents/sentences from disk, so that you never have to hold the entire corpus in RAM at once. This allows you to scale your modeling pipeline up to potentially very large corpora.

In [33]:
unigram_sentences = LineSentence(unigram_sentences_filepath)

Let's take a look at a few sample sentences in our new, transformed file.

In [34]:
for unigram_sentence in it.islice(unigram_sentences, 0, 22):
    print (u' '.join(unigram_sentence))
    print (u'')

world trade organization celebrate 20 year of help global economy grow

geneva jan 1 kuna

the world trade organization world trade organization celebrate thursday the 20th anniversary of its foundation to be the only global international organization deal with the rule of trade between nation

over the past 20 year this organization have on balance make an important contribution to the global economy and to smooth trading relation between nation world trade organization director general roberto azevedo say in a press release

20 year ago on 1 january 1995 the world trade organization open its door for business

since then this organization and the system of transparent multilaterally agree rule that it embody have make a major contribution to the strength and stability of the global economy azevedo say in a press release from the world trade organization headquarters in geneva

he point that the world trade organization over the past two decade have help to boost trade growth resolve 

In [35]:
texts[0]

'world trade organization celebrates 20 years of helping global economy grow. GENEVA, Jan 1 (KUNA)    The World Trade Organization (world trade organization) celebrated Thursday the 20th anniversary of its foundation to be the only global international organization dealing with the rules of trade between nations. Over the past 20 years, this organization has, on balance, made an important contribution to the global economy and to smoother trading relations between nations, world trade organization Director General Roberto Azevedo said in a press release. 20 years ago, on 1 January 1995, the world trade organization opened its doors for business. Since then this organization, and the system of transparent, multilaterally agreed rules that it embodies, has made a major contribution to the strength and stability of the global economy, Azevedo said in a press release from the world trade organization headquarters in Geneva. He pointed that the world trade organization, over the past two de

Next, we'll learn a phrase model that will link individual words into two-word phrases. We'd expect words that together represent a specific concept, like "`ice cream`", to be linked together to form a new, single token: "`ice_cream`".

In [36]:
bigram_model_filepath = os.path.join(intermediate_directory, 'bigram_model_all')

In [37]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to execute modeling yourself.
if 0 == 0:

    bigram_model = Phrases(unigram_sentences)

    bigram_model.save(bigram_model_filepath)
    
# load the finished model from disk
#bigram_model = Phrases.load(bigram_model_filepath)

Wall time: 7.33 s


Now that we have a trained phrase model for word pairs, let's apply it to the review sentences data and explore the results.

In [38]:
bigram_sentences_filepath = os.path.join(intermediate_directory, 'bigram_sentences_all.txt')

In [39]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to execute data prep yourself.
if 0 == 0:

    with codecs.open(bigram_sentences_filepath, 'w', encoding='utf_8') as f:
        for unigram_sentence in unigram_sentences:
            bigram_sentence = u' '.join(bigram_model[unigram_sentence])
            f.write(bigram_sentence + '\n')

Wall time: 15.3 s


In [40]:
bigram_sentences = LineSentence(bigram_sentences_filepath)

In [41]:
for bigram_sentence in it.islice(bigram_sentences, 0, 10):
    print (u' '.join(bigram_sentence))
    print( u'')

world_trade organization celebrate 20 year of help global economy grow

geneva_jan 1 kuna

the world_trade organization world_trade organization celebrate thursday the 20th_anniversary of its foundation to be the only global international organization deal with the rule of trade between nation

over the past 20 year this organization have on balance make an_important contribution to the global economy and to smooth trading relation_between nation world_trade organization_director general_roberto azevedo say in a press_release

20 year_ago on 1 january 1995 the world_trade organization open its door for business

since_then this organization and the system of transparent multilaterally agree rule that it embody have make a major contribution to the strength and stability of the global economy azevedo say in a press_release from the world_trade organization headquarters in geneva

he point that the world_trade organization over the past_two decade have help to boost trade growth resolve 

Looks like the phrase modeling worked! We now see two-word phrases, such as "`ice_cream`" and "`apple_pie`", linked together in the text as a single token. Next, we'll train a _second-order_ phrase model. We'll apply the second-order phrase model on top of the already-transformed data, so that incomplete word combinations like "`vanilla_ice cream`" will become fully joined to "`vanilla_ice_cream`". No disrespect intended to [Vanilla Ice](https://www.youtube.com/watch?v=rog8ou-ZepE), of course.

In [42]:
trigram_model_filepath = os.path.join(intermediate_directory,'trigram_model_all')

In [43]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to execute modeling yourself.
if 0 == 0:
    trigram_model = Phrases(bigram_sentences)
    trigram_model.save(trigram_model_filepath)
    
# load the finished model from disk
#trigram_model = Phrases.load(trigram_model_filepath)

Wall time: 7.76 s


We'll apply our trained second-order phrase model to our first-order transformed sentences, write the results out to a new file, and explore a few of the second-order transformed sentences.

In [44]:
trigram_sentences_filepath = os.path.join(intermediate_directory,'trigram_sentences_all.txt')

In [45]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to execute data prep yourself.
if 0 == 0:

    with codecs.open(trigram_sentences_filepath, 'w', encoding='utf_8') as f:
        
        for bigram_sentence in bigram_sentences:
            
            trigram_sentence = u' '.join(trigram_model[bigram_sentence])
            
            f.write(trigram_sentence + '\n')

Wall time: 15.7 s


In [46]:
trigram_sentences = LineSentence(trigram_sentences_filepath)

In [47]:
for trigram_sentence in it.islice(trigram_sentences, 0, 10):
    print (u' '.join(trigram_sentence))
    print (u'')

world_trade_organization celebrate 20_year of help global_economy grow

geneva_jan 1 kuna

the world_trade_organization world_trade_organization celebrate thursday the 20th_anniversary of its foundation to be the only global international organization deal with the rule of trade between nation

over the past_20 year this organization have on balance make an_important contribution to the global_economy and to smooth trading relation_between nation world_trade_organization_director general_roberto_azevedo say in a press_release

20_year_ago on 1 january 1995 the world_trade_organization open its door for business

since_then this organization and the system of transparent multilaterally agree rule that it embody have make a major contribution to the strength and stability of the global_economy azevedo_say in a press_release from the world_trade_organization headquarters in geneva

he point that the world_trade_organization over the past_two_decade have help to boost trade growth resolve 

Looks like the second-order phrase model was successful. We're now seeing three-word phrases, such as "`vanilla_ice_cream`" and "`cinnamon_ice_cream`".

The final step of our text preparation process circles back to the complete text of the reviews. We're going to run the complete text of the reviews through a pipeline that applies our text normalization and phrase models.

In addition, we'll remove stopwords at this point. _Stopwords_ are very common words, like _a_, _the_, _and_, and so on, that serve functional roles in natural language, but typically don't contribute to the overall meaning of text. Filtering stopwords is a common procedure that allows higher-level NLP modeling techniques to focus on the words that carry more semantic weight.

Finally, we'll write the transformed text out to a new file, with one review per line.

In [48]:
trigram_reviews_filepath = os.path.join(intermediate_directory,'trigram_transformed_reviews_all.txt')

In [50]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to execute data prep yourself.
if 0 == 0:

    with codecs.open(trigram_reviews_filepath, 'w', encoding='utf_8') as f:
        
        for parsed_review in nlp.pipe(line_review(review_txt_filepath),
                                      batch_size=10000, n_threads=4):
            
            # lemmatize the text, removing punctuation and whitespace
            unigram_review = [token.lemma_ for token in parsed_review
                              if not punct_space(token)]
            
            # apply the first-order and second-order phrase models
            bigram_review = bigram_model[unigram_review]
            trigram_review = trigram_model[bigram_review]
            
            # remove any remaining stopwords
            trigram_review = [term for term in trigram_review
                              if term not in spacy.en.STOPWORDS]
            
            # write the transformed review as a line in the new file
            trigram_review = u' '.join(trigram_review)
            f.write(trigram_review + '\n')

Wall time: 4min 33s


In [53]:
print(u'Original:' + u'\n')

for review in it.islice(line_review(review_txt_filepath), 0, 1):
    print(review)

print(u'----' + u'\n')
print(u'Transformed:' + u'\n')

with codecs.open(trigram_reviews_filepath, encoding='utf_8') as f:
    for review in it.islice(f, 0, 1):
        print(review) 

Original:

world trade organization celebrates 20 years of helping global economy grow. GENEVA, Jan 1 (KUNA)    The World Trade Organization (world trade organization) celebrated Thursday the 20th anniversary of its foundation to be the only global international organization dealing with the rules of trade between nations. Over the past 20 years, this organization has, on balance, made an important contribution to the global economy and to smoother trading relations between nations, world trade organization Director General Roberto Azevedo said in a press release. 20 years ago, on 1 January 1995, the world trade organization opened its doors for business. Since then this organization, and the system of transparent, multilaterally agreed rules that it embodies, has made a major contribution to the strength and stability of the global economy, Azevedo said in a press release from the world trade organization headquarters in Geneva. He pointed that the world trade organization, over the p

Let's preview the results. We'll grab one review from the file with the original, untransformed text, grab the same review from the file with the normalized and transformed text, and compare the two.

You can see that most of the grammatical structure has been scrubbed from the text &mdash; capitalization, articles/conjunctions, punctuation, spacing, etc. However, much of the general semantic *meaning* is still present. Also, multi-word concepts such as "`friday_night`" and "`above_average`" have been joined into single tokens, as expected. The review text is now ready for higher-level modeling. 

## Topic Modeling with Latent Dirichlet Allocation

*Topic modeling* is family of techniques that can be used to describe and summarize the documents in a corpus according to a set of latent "topics". For this demo, we'll be using [*Latent Dirichlet Allocation*](http://www.jmlr.org/papers/volume3/blei03a/blei03a.pdf) or LDA, a popular approach to topic modeling.

In many conventional NLP applications, documents are represented a mixture of the individual tokens (words and phrases) they contain. In other words, a document is represented as a *vector* of token counts. There are two layers in this model &mdash; documents and tokens &mdash; and the size or dimensionality of the document vectors is the number of tokens in the corpus vocabulary. This approach has a number of disadvantages:
* Document vectors tend to be large (one dimension for each token $\Rightarrow$ lots of dimensions)
* They also tend to be very sparse. Any given document only contains a small fraction of all tokens in the vocabulary, so most values in the document's token vector are 0.
* The dimensions are fully indepedent from each other &mdash; there's no sense of connection between related tokens, such as _knife_ and _fork_.

LDA injects a third layer into this conceptual model. Documents are represented as a mixture of a pre-defined number of *topics*, and the *topics* are represented as a mixture of the individual tokens in the vocabulary. The number of topics is a model hyperparameter selected by the practitioner. LDA makes a prior assumption that the (document, topic) and (topic, token) mixtures follow [*Dirichlet*](https://en.wikipedia.org/wiki/Dirichlet_distribution) probability distributions. This assumption encourages documents to consist mostly of a handful of topics, and topics to consist mostly of a modest set of the tokens.

![LDA](https://s3.amazonaws.com/skipgram-images/LDA.png)

LDA is fully unsupervised. The topics are "discovered" automatically from the data by trying to maximize the likelihood of observing the documents in your corpus, given the modeling assumptions. They are expected to capture some latent structure and organization within the documents, and often have a meaningful human interpretation for people familiar with the subject material.

We'll again turn to gensim to assist with data preparation and modeling. In particular, gensim offers a high-performance parallelized implementation of LDA with its [**LdaMulticore**](https://radimrehurek.com/gensim/models/ldamulticore.html) class.

In [54]:
from gensim.corpora import Dictionary, MmCorpus
from gensim.models.ldamulticore import LdaMulticore

import pyLDAvis
import pyLDAvis.gensim
import warnings
# import cPickle as pickle
# import _pickle as cPickle

C:\Program Files\Anaconda3\lib\site-packages\funcy\decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
C:\Program Files\Anaconda3\lib\site-packages\funcy\decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
C:\Program Files\Anaconda3\lib\site-packages\funcy\decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
C:\Program Files\Anaconda3\lib\site-packages\funcy\decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
C:\Program Files\Anaconda3\lib\site-packages\funcy\decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
C:\Program Files\Anaconda3\lib\site-packages\

The first step to creating an LDA model is to learn the full vocabulary of the corpus to be modeled. We'll use gensim's [**Dictionary**](https://radimrehurek.com/gensim/corpora/dictionary.html) class for this.

In [55]:
trigram_dictionary_filepath = os.path.join(intermediate_directory,
                                           'trigram_dict_all.dict')

In [56]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to learn the dictionary yourself.
if 0 == 0:

    trigram_reviews = LineSentence(trigram_reviews_filepath)

    # learn the dictionary by iterating over all of the reviews
    trigram_dictionary = Dictionary(trigram_reviews)
    
    # filter tokens that are very rare or too common from
    # the dictionary (filter_extremes) and reassign integer ids (compactify)
    #trigram_dictionary.filter_extremes(no_below=10, no_above=0.4)
    trigram_dictionary.filter_extremes(no_below=10, no_above=0.8)
    trigram_dictionary.compactify()

    trigram_dictionary.save(trigram_dictionary_filepath)

Wall time: 2.73 s


In [57]:
    
# load the finished dictionary from disk
trigram_dictionary = Dictionary.load(trigram_dictionary_filepath)

Like many NLP techniques, LDA uses a simplifying assumption known as the [*bag-of-words* model](https://en.wikipedia.org/wiki/Bag-of-words_model). In the bag-of-words model, a document is represented by the counts of distinct terms that occur within it. Additional information, such as word order, is discarded. 

Using the gensim Dictionary we learned to generate a bag-of-words representation for each review. The `trigram_bow_generator` function implements this. We'll save the resulting bag-of-words reviews as a matrix.

In the following code, "bag-of-words" is abbreviated as `bow`.

In [58]:
trigram_bow_filepath = os.path.join(intermediate_directory,
                                    'trigram_bow_corpus_all.mm')

In [59]:
def trigram_bow_generator(filepath):
    """
    generator function to read reviews from a file
    and yield a bag-of-words representation
    """
    
    for review in LineSentence(filepath):
        yield trigram_dictionary.doc2bow(review)

In [60]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to build the bag-of-words corpus yourself.
if 0 == 0:

    # generate bag-of-words representations for
    # all reviews and save them as a matrix
    MmCorpus.serialize(trigram_bow_filepath,
                       trigram_bow_generator(trigram_reviews_filepath))
    

Wall time: 3.98 s


In [62]:
# load the finished bag-of-words corpus from disk
trigram_bow_corpus = MmCorpus(trigram_bow_filepath)

With the bag-of-words corpus, we're finally ready to learn our topic model from the reviews. We simply need to pass the bag-of-words matrix and Dictionary from our previous steps to `LdaMulticore` as inputs, along with the number of topics the model should learn. For this demo, we're asking for 50 topics.

In [63]:
lda_model_filepath = os.path.join(intermediate_directory, 'lda_model_all')

In [69]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to train the LDA model yourself.
if 0 == 0:

    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        
        # workers => sets the parallelism, and should be
        # set to your number of physical cores minus one
        lda = LdaMulticore(trigram_bow_corpus,
                           num_topics=5,
                           id2word=trigram_dictionary,
                           workers=3)
    
    lda.save(lda_model_filepath)
    

Wall time: 12.8 s


In [70]:
# load the finished LDA model from disk
lda = LdaMulticore.load(lda_model_filepath)

Our topic model is now trained and ready to use! Since each topic is represented as a mixture of tokens, you can manually inspect which tokens have been grouped together into which topics to try to understand the patterns the model has discovered in the data.

In [71]:
def explore_topic(topic_number, topn=25):
    """
    accept a user-supplied topic number and
    print out a formatted list of the top terms
    """
        
    print (u'{:20} {}'.format(u'term', u'frequency') + u'\n')

    for term, frequency in lda.show_topic(topic_number, topn=25):
        print (u'{:20} {:.3f}'.format(term, round(frequency, 3)))

In [72]:
explore_topic(topic_number=0)

term                 frequency

trade                0.022
world_trade_organization 0.012
india                0.008
china                0.008
united_states        0.007
country              0.007
eu                   0.005
agreement            0.005
new                  0.005
deal                 0.005
global               0.005
export               0.004
government           0.004
market               0.003
issue                0.003
include              0.003
member               0.003
negotiation          0.003
international        0.003
world                0.003
import               0.003
economy              0.003
support              0.003
policy               0.003
economic             0.003


The first topic has strong associations with words like *taco*, *salsa*, *chip*, *burrito*, and *margarita*, as well as a handful of more general words. You might call this the **Mexican food** topic!

It's possible to go through and inspect each topic in the same way, and try to assign a human-interpretable label that captures the essence of each one. I've given it a shot for all 50 topics below.

In [70]:
topic_names = {0: u'mexican',
               1: u'menu',
               2: u'thai',
               3: u'steak',
               4: u'donuts & appetizers',
               5: u'specials',
               6: u'soup',
               7: u'wings, sports bar',
               8: u'foreign language',
               9: u'las vegas',
               10: u'chicken',
               11: u'aria buffet',
               12: u'noodles',
               13: u'ambience & seating',
               14: u'sushi',
               15: u'arizona',
               16: u'family',
               17: u'price',
               18: u'sweet',
               19: u'waiting',
               20: u'general',
               21: u'tapas',
               22: u'dirty',
               23: u'customer service',
               24: u'restrooms',
               25: u'chinese',
               26: u'gluten free',
               27: u'pizza',
               28: u'seafood',
               29: u'amazing',
               30: u'eat, like, know, want',
               31: u'bars',
               32: u'breakfast',
               33: u'location & time',
               34: u'italian',
               35: u'barbecue',
               36: u'arizona',
               37: u'indian',
               38: u'latin & cajun',
               39: u'burger & fries',
               40: u'vegetarian',
               41: u'lunch buffet',
               42: u'customer service',
               43: u'taco, ice cream',
               44: u'high cuisine',
               45: u'healthy',
               46: u'salad & sandwich',
               47: u'greek',
               48: u'poor experience',
               49: u'wine & dine'}

In [71]:
import pickle

topic_names_filepath = os.path.join(intermediate_directory, 'topic_names.pkl')


with open(topic_names_filepath, 'wb') as f:
    pickle.dump(topic_names, f)

You can see that, along with **mexican**, there are a variety of topics related to different styles of food, such as **thai**, **steak**, **sushi**, **pizza**, and so on. In addition, there are topics that are more related to the overall restaurant *experience*, like **ambience & seating**, **good service**, **waiting**, and **price**.

Beyond these two categories, there are still some topics that are difficult to apply a meaningful human interpretation to, such as topic 30 and 43.

Manually reviewing the top terms for each topic is a helpful exercise, but to get a deeper understanding of the topics and how they relate to each other, we need to visualize the data &mdash; preferably in an interactive format. Fortunately, we have the fantastic [**pyLDAvis**](https://pyldavis.readthedocs.io/en/latest/readme.html) library to help with that!

pyLDAvis includes a one-line function to take topic models created with gensim and prepare their data for visualization.

In [73]:
LDAvis_data_filepath = os.path.join(intermediate_directory, 'ldavis_prepared')

In [74]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to execute data prep yourself.
if 0 == 0:

    LDAvis_prepared = pyLDAvis.gensim.prepare(lda, trigram_bow_corpus,
                                              trigram_dictionary)

#     with open(LDAvis_data_filepath, 'wb') as f:
#         pickle.dump(LDAvis_prepared, f)
        
# # load the pre-prepared pyLDAvis data from disk
# with open(LDAvis_data_filepath) as f:
#     LDAvis_prepared = pickle.load(f)

Wall time: 23.1 s


`pyLDAvis.display(...)` displays the topic model visualization in-line in the notebook.

In [75]:
pyLDAvis.display(LDAvis_prepared)

In [59]:
# pyLDAvis.show(LDAvis_prepared)

### Wait, what am I looking at again?
There are a lot of moving parts in the visualization. Here's a brief summary:

* On the left, there is a plot of the "distance" between all of the topics (labeled as the _Intertopic Distance Map_)
  * The plot is rendered in two dimensions according a [*multidimensional scaling (MDS)*](https://en.wikipedia.org/wiki/Multidimensional_scaling) algorithm. Topics that are generally similar should be appear close together on the plot, while *dis*similar topics should appear far apart.
  * The relative size of a topic's circle in the plot corresponds to the relative frequency of the topic in the corpus.
  * An individual topic may be selected for closer scrutiny by clicking on its circle, or entering its number in the "selected topic" box in the upper-left.
* On the right, there is a bar chart showing top terms.
  * When no topic is selected in the plot on the left, the bar chart shows the top-30 most "salient" terms in the corpus. A term's *saliency* is a measure of both how frequent the term is in the corpus and how "distinctive" it is in distinguishing between different topics.
  * When a particular topic is selected, the bar chart changes to show the top-30 most "relevant" terms for the selected topic. The relevance metric is controlled by the parameter $\lambda$, which can be adjusted with a slider above the bar chart.
    * Setting the $\lambda$ parameter close to 1.0 (the default) will rank the terms solely according to their probability within the topic.
    * Setting $\lambda$ close to 0.0 will rank the terms solely according to their "distinctiveness" or "exclusivity" within the topic &mdash; i.e., terms that occur *only* in this topic, and do not occur in other topics.
    * Setting $\lambda$ to values between 0.0 and 1.0 will result in an intermediate ranking, weighting term probability and exclusivity accordingly.
* Rolling the mouse over a term in the bar chart on the right will cause the topic circles to resize in the plot on the left, to show the strength of the relationship between the topics and the selected term.

A more detailed explanation of the pyLDAvis visualization can be found [here](https://cran.r-project.org/web/packages/LDAvis/vignettes/details.pdf). Unfortunately, though the data used by gensim and pyLDAvis are the same, they don't use the same ID numbers for topics. If you need to match up topics in gensim's `LdaMulticore` object and pyLDAvis' visualization, you have to dig through the terms manually.

### Analyzing our LDA model
The interactive visualization pyLDAvis produces is helpful for both:
1. Better understanding and interpreting individual topics, and
1. Better understanding the relationships between the topics.

For (1), you can manually select each topic to view its top most freqeuent and/or "relevant" terms, using different values of the $\lambda$ parameter. This can help when you're trying to assign a human interpretable name or "meaning" to each topic.

For (2), exploring the _Intertopic Distance Plot_ can help you learn about how topics relate to each other, including potential higher-level structure between groups of topics.

In our plot, there is a stark divide along the x-axis, with two topics far to the left and most of the remaining 48 far to the right. Inspecting the two outlier topics provides a plausible explanation: both topics contain many non-English words, while most of the rest of the topics are in English. So, one of the main attributes that distinguish the reviews in the dataset from one another is their language.

This finding isn't entirely a surprise. In addition to English-speaking cities, the Yelp dataset includes reviews of businesses in Montreal and Karlsruhe, Germany, often written in French and German, respectively. Multiple languages isn't a problem for our demo, but for a real NLP application, you might need to ensure that the text you're processing is written in English (or is at least tagged for language) before passing it along to some downstream processing. If that were the case, the divide along the x-axis in the topic plot would immediately alert you to a potential data quality issue.

The y-axis separates two large groups of topics &mdash; let's call them "super-topics" &mdash; one in the upper-right quadrant and the other in the lower-right quadrant. These super-topics correlate reasonably well with the pattern we'd noticed while naming the topics:
* The super-topic in the *lower*-right tends to be about *food*. It groups together the **burger & fries**, **breakfast**, **sushi**, **barbecue**, and **greek** topics, among others.
* The super-topic in the *upper*-right tends to be about other elements of the *restaurant experience*. It groups together the **ambience & seating**, **location & time**, **family**, and **customer service** topics, among others.

So, in addition to the 50 direct topics the model has learned, our analysis suggests a higher-level pattern in the data. Restaurant reviewers in the Yelp dataset talk about two main things in their reviews, in general: (1) the food, and (2) their overall restaurant experience. For this dataset, this is a very intuitive result, and we probably didn't need a sophisticated modeling technique to tell it to us. When working with datasets from other domains, though, such high-level patterns may be much less obvious from the outset &mdash; and that's where topic modeling can help.

### Describing text with LDA
Beyond data exploration, one of the key uses for an LDA model is providing a compact, quantitative description of natural language text. Once an LDA model has been trained, it can be used to represent free text as a mixture of the topics the model learned from the original corpus. This mixture can be interpreted as a probability distribution across the topics, so the LDA representation of a paragraph of text might look like 50% _Topic A_, 20% _Topic B_, 20% _Topic C_, and 10% _Topic D_.

To use an LDA model to generate a vector representation of new text, you'll need to apply any text preprocessing steps you used on the model's training corpus to the new text, too. For our model, the preprocessing steps we used include:
1. Using spaCy to remove punctuation and lemmatize the text
1. Applying our first-order phrase model to join word pairs
1. Applying our second-order phrase model to join longer phrases
1. Removing stopwords
1. Creating a bag-of-words representation

Once you've applied these preprocessing steps to the new text, it's ready to pass directly to the model to create an LDA representation. The `lda_description(...)` function will perform all these steps for us, including printing the resulting topical description of the input text.

In [75]:
def get_sample_review(review_number):
    """
    retrieve a particular review index
    from the reviews file and return it
    """
    
    return list(it.islice(line_review(review_txt_filepath),
                          review_number, review_number+1))[0]

In [76]:
def lda_description(review_text, min_topic_freq=0.05):
    """
    accept the original text of a review and (1) parse it with spaCy,
    (2) apply text pre-proccessing steps, (3) create a bag-of-words
    representation, (4) create an LDA representation, and
    (5) print a sorted list of the top topics in the LDA representation
    """
    
    # parse the review text with spaCy
    parsed_article = nlp(review_text)
    
    # lemmatize the text and remove punctuation and whitespace
    unigram_review = [token.lemma_ for token in parsed_article
                      if not punct_space(token)]
    
    # apply the first-order and secord-order phrase models
    bigram_review = bigram_model[unigram_review]
    trigram_review = trigram_model[bigram_review]
    
    # remove any remaining stopwords
    trigram_review = [term for term in trigram_review
                      if not term in spacy.en.STOPWORDS]
    
    # create a bag-of-words representation
    review_bow = trigram_dictionary.doc2bow(trigram_review)
    
    # create an LDA representation
    review_lda = lda[review_bow]
    
    # sort with the most highly related topics first
    review_lda = sorted(review_lda, key=lambda (topic_number, freq): -freq)
    
    for topic_number, freq in review_lda:
        if freq < min_topic_freq:
            break
            
        # print the most highly related topic names and frequencies
        print '{:25} {}'.format(topic_names[topic_number],
                                round(freq, 3))

SyntaxError: invalid syntax (<ipython-input-76-b08efbaf18cd>, line 31)

In [77]:
sample_review = get_sample_review(50)
print sample_review

SyntaxError: Missing parentheses in call to 'print' (<ipython-input-77-b73c848a03f6>, line 2)

In [55]:
lda_description(sample_review)

breakfast                 0.4
amazing                   0.341
customer service          0.192


In [56]:
sample_review = get_sample_review(100)
print sample_review

I went there for dinner last night with a client.  This is second time I visited.  I had a scotch and he had a Guiness.  The (-1) is for drink selection. Just stock some better beers and higher end scotch and you're five stars.

We started with the meatballs covered with Provolone and other blessed goodness.  These did not disappoint.  I had a four cheese pizza with sweet sausage and garlic.  It was fantastic.  They have so many good dishes, but I wanted the pizza last night.  I couldn't finish the pizza - way to go big medium pizza.

I finished up with a coffee.  The parking can be a bit of a challenge on the street, but it's a small town atmosphere in Carnegie, PA.  I love the downtown there.



In [57]:
lda_description(sample_review)

pizza                     0.28
bars                      0.193
amazing                   0.172
wine & dine               0.149


## Word Vector Embedding with Word2Vec

Pop quiz! Can you complete this text snippet?

<br><br>

![word2vec quiz](https://s3.amazonaws.com/skipgram-images/word2vec-1.png)

<br><br><br>
You just demonstrated the core machine learning concept behind word vector embedding models!
<br><br><br>

![word2vec quiz 2](https://s3.amazonaws.com/skipgram-images/word2vec-2.png)

The goal of *word vector embedding models*, or *word vector models* for short, is to learn dense, numerical vector representations for each term in a corpus vocabulary. If the model is successful, the vectors it learns about each term should encode some information about the *meaning* or *concept* the term represents, and the relationship between it and other terms in the vocabulary. Word vector models are also fully unsupervised &mdash they learn all of these meanings and relationships solely by analyzing the text of the corpus, without any advance knowledge provided.

Perhaps the best-known word vector model is [word2vec](https://arxiv.org/pdf/1301.3781v3.pdf), originally proposed in 2013. The general idea of word2vec is, for a given *focus word*, to use the *context* of the word &mdash; i.e., the other words immediately before and after it &mdash; to provide hints about what the focus word might mean. To do this, word2vec uses a *sliding window* technique, where it considers snippets of text only a few tokens long at a time.

At the start of the learning process, the model initializes random vectors for all terms in the corpus vocabulary. The model then slides the window across every snippet of text in the corpus, with each word taking turns as the focus word. Each time the model considers a new snippet, it tries to learn some information about the focus word based on the surrouding context, and it "nudges" the words' vector representations accordingly. One complete pass sliding the window across all of the corpus text is known as a training *epoch*. It's common to train a word2vec model for multiple passes/epochs over the corpus. Over time, the model rearranges the terms' vector representations such that terms that frequently appear in similar contexts have vector representations that are *close* to each other in vector space.

For a deeper dive into word2vec's machine learning process, see [here](https://arxiv.org/pdf/1411.2738v4.pdf).

Word2vec has a number of user-defined hyperparameters, including:
- The dimensionality of the vectors. Typical choices include a few dozen to several hundred.
- The width of the sliding window, in tokens. Five is a common default choice, but narrower and wider windows are possible.
- The number of training epochs.

For using word2vec in Python, [gensim](https://rare-technologies.com/deep-learning-with-word2vec-and-gensim/) comes to the rescue again! It offers a [highly-optimized](https://rare-technologies.com/word2vec-in-python-part-two-optimizing/), [parallelized](https://rare-technologies.com/parallelizing-word2vec-in-python/) implementation of the word2vec algorithm with its [Word2Vec](https://radimrehurek.com/gensim/models/word2vec.html) class.

In [78]:
from gensim.models import Word2Vec

trigram_sentences = LineSentence(trigram_sentences_filepath)
word2vec_filepath = os.path.join(intermediate_directory, 'word2vec_model_all')

We'll train our word2vec model using the normalized sentences with our phrase models applied. We'll use 100-dimensional vectors, and set up our training process to run for twelve epochs.

In [79]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to train the word2vec model yourself.
if 0 == 0:

    # initiate the model and perform the first epoch of training
    food2vec = Word2Vec(trigram_sentences, size=100, window=5,
                        min_count=20, sg=1, workers=4)
    
    food2vec.save(word2vec_filepath)

    # perform another 11 epochs of training
    for i in range(1,12):

        food2vec.train(trigram_sentences)
        food2vec.save(word2vec_filepath)
        
# load the finished model from disk
food2vec = Word2Vec.load(word2vec_filepath)
food2vec.init_sims()

print (u'{} training epochs so far.'.format(food2vec.train_count))

12 training epochs so far.
Wall time: 6min 21s


On my four-core machine, each epoch over all the text in the ~1 million Yelp reviews takes about 5-10 minutes.

In [80]:
print (u'{:,} terms in the food2vec vocabulary.'.format(len(food2vec.vocab)))

9,753 terms in the food2vec vocabulary.


Let's take a peek at the word vectors our model has learned. We'll create a pandas DataFrame with the terms as the row labels, and the 100 dimensions of the word vector model as the columns.

In [81]:
# build a list of the terms, integer indices,
# and term counts from the food2vec model vocabulary
ordered_vocab = [(term, voc.index, voc.count)
                 for term, voc in food2vec.vocab.items()]

# sort by the term counts, so the most common terms appear first
# ordered_vocab = sorted(ordered_vocab, key=lambda(term, index, count): -count)

# unzip the terms, integer indices, and counts into separate lists
ordered_terms, term_indices, term_counts = zip(*ordered_vocab)

# create a DataFrame with the food2vec vectors as data,
# and the terms as row labels
word_vectors = pd.DataFrame(food2vec.syn0norm[term_indices, :],
                            index=ordered_terms)

word_vectors

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
bus,-0.080024,0.014366,0.104225,-0.001783,0.138101,-0.020336,-0.052383,0.070011,-0.146251,0.044949,...,-0.126520,0.164104,-0.061736,0.155355,0.042468,-0.105244,-0.091141,0.003853,-0.017970,-0.135068
his_colleague,-0.042075,0.030082,-0.075944,-0.074296,0.066482,0.184562,-0.134259,0.046688,-0.041900,-0.034023,...,-0.051961,0.129457,0.019394,0.122246,0.079234,-0.066176,-0.147022,-0.010835,-0.030061,-0.037209
alongside,0.052920,-0.099577,0.015531,-0.014609,0.061962,-0.178176,-0.196008,-0.047763,-0.043148,0.102216,...,0.098211,0.129959,0.177143,0.044355,0.017987,-0.156203,-0.046525,0.124704,0.163587,0.002060
guideline,-0.187728,-0.034365,-0.054225,-0.136765,-0.190447,-0.197913,0.052316,-0.031813,-0.099282,-0.053119,...,0.032284,0.060387,0.008700,-0.003504,-0.097161,-0.004394,-0.013445,0.053867,-0.065558,-0.233144
frankly,-0.196406,-0.031395,0.040069,-0.028096,0.034187,0.115287,-0.078679,0.031826,-0.217811,-0.057959,...,-0.134207,-0.103379,0.067337,-0.043022,-0.173540,0.128404,-0.159845,-0.106545,0.061488,0.034640
2001,-0.080314,0.019702,-0.075511,-0.022298,0.055174,-0.035094,0.017063,0.066161,0.002830,-0.102521,...,0.096271,-0.016305,0.094436,0.009550,0.023332,-0.025583,-0.023907,0.001143,0.017951,-0.157098
attribute,0.222525,0.185559,0.119894,-0.043786,-0.051014,-0.109519,-0.106034,-0.001784,-0.006567,0.063550,...,-0.024092,0.071966,-0.161996,-0.014606,0.162994,-0.024900,0.068904,-0.132208,-0.034711,-0.176710
fast,0.018810,0.163287,-0.003456,-0.036305,-0.097950,-0.063989,0.011651,-0.018093,-0.049833,0.060482,...,-0.074680,0.070363,0.191779,-0.114466,0.014564,0.086850,0.068816,-0.047692,-0.050511,0.075380
rtas,0.106760,-0.051138,-0.041242,0.067789,0.156094,-0.048995,-0.001513,-0.141543,-0.132410,0.070046,...,-0.053633,-0.074809,0.034628,0.079852,0.028315,-0.049808,0.034093,0.024389,-0.036169,-0.118299
interestingly,-0.049877,0.020517,-0.011597,-0.049590,-0.083755,0.099211,-0.134028,0.154634,-0.006640,0.005523,...,-0.038973,-0.027939,0.080848,-0.107094,-0.014427,0.057790,-0.156068,0.050837,-0.051017,0.006609


Holy wall of numbers! This DataFrame has 50,835 rows &mdash; one for each term in the vocabulary &mdash; and 100 colums. Our model has learned a quantitative vector representation for each term, as expected.

Put another way, our model has "embedded" the terms into a 100-dimensional vector space.

### So... what can we do with all these numbers?
The first thing we can use them for is to simply look up related words and phrases for a given term of interest.

In [134]:
def get_related_terms(token, topn=10):
    """
    look up the topn most similar terms to token
    and print them as a formatted list
    """

    for word, similarity in food2vec.most_similar(positive=[token], topn=topn):

        print (u'{:20} {}'.format(word, round(similarity, 3)))

### What things are like Burger King?

In [64]:
get_related_terms(u'burger_king')

mcdonalds            0.895
wendy_'s             0.855
mcd_'s               0.853
mcdonald_'s          0.852
denny_'s             0.816
bk                   0.808
carl_'s_jr.          0.8
red_robin            0.792
mickey_d_'s          0.771
sonic                0.765


In [149]:
get_related_terms(u'ita')

information_technology_agreement_ita 0.71
information_technology_agreement 0.697
product_scope        0.649
ega                  0.615
plurilateral         0.574
environmental_goods_agreement 0.574
cut_tariff           0.554
201_product          0.539
tariff_cutting       0.522
information_technology 0.52


The model has learned that fast food restaurants are similar to each other! In particular, *mcdonalds* and *wendy's* are the most similar to Burger King, according to this dataset. In addition, the model has found that alternate spellings for the same entities are probably related, such as *mcdonalds*, *mcdonald's* and *mcd's*.

### When is happy hour?

In [65]:
get_related_terms(u'happy_hour')

hh                   0.874
reverse_happy_hour   0.801
happy_hr             0.771
during_happy_hour    0.672
mon_fri              0.634
3_6pm                0.632
hh.                  0.631
4_7pm                0.625
special              0.621
happy_hour_3_6pm     0.618


The model has noticed several alternate spellings for happy hour, such as *hh* and *happy hr*, and assesses them as highly related. If you were looking for reviews about happy hour, such alternate spellings would be very helpful to know.

Taking a deeper look &mdash; the model has turned up phrases like *3-6pm*, *4-7pm*, and *mon-fri*, too. This is especially interesting, because the model has no advance knowledge at all about what happy hour is, and what time of day it should be. But simply by scanning through restaurant reviews, the model has discovered that the concept of happy hour has something very important to do with that block of time around 3-7pm on weekdays.

### Let's make pasta tonight. Which style do you want?

In [66]:
get_related_terms(u'pasta', topn=20)

lasagna              0.798
spaghetti            0.773
bolognese            0.757
fettuccine           0.748
penne                0.745
rigatoni             0.743
angel_hair           0.721
linguine             0.716
angel_hair_pasta     0.712
penne_pasta          0.712
carbonara            0.705
fettucini            0.704
alfredo              0.703
tortellini           0.703
manicotti            0.7
ziti                 0.698
gnocci               0.694
ravioli              0.694
linguini             0.693
risotto              0.691


## Word algebra!
No self-respecting word2vec demo would be complete without a healthy dose of *word algebra*, also known as *analogy completion*.

The core idea is that once words are represented as numerical vectors, you can do math with them. The mathematical procedure goes like this:
1. Provide a set of words or phrases that you'd like to add or subtract.
1. Look up the vectors that represent those terms in the word vector model.
1. Add and subtract those vectors to produce a new, combined vector.
1. Look up the most similar vector(s) to this new, combined vector via cosine similarity.
1. Return the word(s) associated with the similar vector(s).

But more generally, you can think of the vectors that represent each word as encoding some information about the *meaning* or *concepts* of the word. What happens when you ask the model to combine the meaning and concepts of words in new ways? Let's see.

In [151]:
def word_algebra(add=[], subtract=[], topn=1):
    """
    combine the vectors associated with the words provided
    in add= and subtract=, look up the topn most similar
    terms to the combined vector, and print the result(s)
    """
    answers = food2vec.most_similar(positive=add, negative=subtract, topn=topn)
    
    for term, similarity in answers:
        print (term)

### breakfast + lunch = ?
Let's start with a softball.

In [68]:
word_algebra(add=[u'breakfast', u'lunch'])

brunch


OK, so the model knows that *brunch* is a combination of *breakfast* and *lunch*. What else?

### lunch - day + night = ?

In [156]:
word_algebra(add=[u'beijing', u'china'], subtract=[u'eu'])

chinas


Now we're getting a bit more nuanced. The model has discovered that:
- Both *lunch* and *dinner* are meals
- The main difference between them is time of day
- Day and night are times of day
- Lunch is associated with day, and dinner is associated with night

What else?

### taco - mexican + chinese = ?

In [70]:
word_algebra(add=[u'taco', u'chinese'], subtract=[u'mexican'])

dumpling


Here's an entirely new and different type of relationship that the model has learned.
- It knows that tacos are a characteristic example of Mexican food
- It knows that Mexican and Chinese are both styles of food
- If you subtract *Mexican* from *taco*, you're left with something like the concept of a *"characteristic type of food"*, which is represented as a new vector
- If you add that new *"characteristic type of food"* vector to Chinese, you get *dumpling*.

What else?

### bun - american + mexican = ?

In [71]:
word_algebra(add=[u'bun', u'mexican'], subtract=[u'american'])

tortilla


The model knows that both *buns* and *tortillas* are the doughy thing that goes on the outside of your real food, and that the primary difference between them is the style of food they're associated with.

What else?

### filet mignon - beef + seafood = ?

In [72]:
word_algebra(add=[u'filet_mignon', u'seafood'], subtract=[u'beef'])

raw_oyster


The model has learned a concept of *delicacy*. If you take filet mignon and subtract beef from it, you're left with a vector that roughly corresponds to delicacy. If you add the delicacy vector to *seafood*, you get *raw oyster*.

What else?

### coffee - drink + snack = ?

In [73]:
word_algebra(add=[u'coffee', u'snack'], subtract=[u'drink'])

pastry


The model knows that if you're on your coffee break, but instead of drinking something, you're eating something... that thing is most likely a pastry.

What else?

### Burger King + fine dining = ?

In [74]:
word_algebra(add=[u'burger_king', u'fine_dining'])

denny_'s


Touché. It makes sense, though. The model has learned that both Burger King and Denny's are large chains, and that both serve fast, casual, American-style food. But Denny's has some elements that are slightly more upscale, such as printed menus and table service. Fine dining, indeed.

*What if we keep going?*

### Denny's + fine dining = ?

In [75]:
word_algebra(add=[u"denny_'s", u'fine_dining'])

applebee_'s


This seems like a good place to land... what if we explore the vector space around *Applebee's* a bit, in a few different directions? Let's see what we find.

#### Applebee's + italian = ?

In [76]:
word_algebra(add=[u"applebee_'s", u'italian'])

olive_garden


#### Applebee's + pancakes = ?

In [77]:
word_algebra(add=[u"applebee_'s", u'pancakes'])

ihop


#### Applebee's + pizza = ?

In [78]:
word_algebra(add=[u"applebee_'s", u'pizza'])

pizza_hut


You could do this all day. One last analogy before we move on...

### wine - grapes + barley = ?

In [79]:
word_algebra(add=[u'wine', u'barley'], subtract=[u'grapes'])

beer


## Word Vector Visualization with t-SNE

[t-Distributed Stochastic Neighbor Embedding](https://lvdmaaten.github.io/publications/papers/JMLR_2008.pdf), or *t-SNE* for short, is a dimensionality reduction technique to assist with visualizing high-dimensional datasets. It attempts to map high-dimensional data onto a low two- or three-dimensional representation such that the relative distances between points are preserved as closely as possible in both high-dimensional and low-dimensional space.

scikit-learn provides a convenient implementation of the t-SNE algorithm with its [TSNE](http://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html) class.

In [157]:
from sklearn.manifold import TSNE

Our input for t-SNE will be the DataFrame of word vectors we created before. Let's first:
1. Drop stopwords &mdash; it's probably not too interesting to visualize *the*, *of*, *or*, and so on
1. Take only the 5,000 most frequent terms in the vocabulary &mdash; no need to visualize all ~50,000 terms right now.

In [158]:
tsne_input = word_vectors.drop(spacy.en.STOPWORDS, errors=u'ignore')
tsne_input = tsne_input.head(5000)

In [159]:
tsne_input.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
next_two,0.014245,0.081925,0.120588,0.036345,-0.098385,0.079087,0.034834,-0.098878,-0.087057,0.254529,...,0.221620,-0.064378,-0.067119,-0.036841,-0.121077,0.165621,-0.115393,-0.073094,-0.134284,-0.147702
surrogate,0.007651,0.030414,0.063372,0.117383,-0.109927,0.034824,0.002840,-0.024183,-0.169811,-0.080141,...,0.086356,0.106699,0.215246,-0.025896,0.061038,-0.072053,0.063339,-0.024687,0.175213,-0.131541
tradition,0.183083,-0.164399,-0.110694,-0.114304,-0.032261,-0.050025,0.006540,0.033276,-0.071628,0.198329,...,-0.028671,-0.070223,0.070406,0.049410,-0.115393,-0.122527,0.123649,0.120971,-0.048232,0.079311
rio,0.116441,0.057626,-0.048533,0.160459,-0.002820,-0.064385,0.252695,-0.055748,-0.009119,0.011890,...,0.146543,0.140933,-0.021025,0.113526,-0.080147,0.083371,0.137117,-0.074767,-0.061146,-0.048244
gm_food,0.099589,0.019368,-0.104975,-0.157651,-0.122692,0.039481,-0.038713,0.003003,-0.105402,0.070464,...,0.021812,0.062145,0.064791,-0.072656,-0.091894,-0.064003,-0.017857,0.060207,0.107875,-0.033575


In [160]:
tsne_filepath = os.path.join(intermediate_directory,
                             u'tsne_model')

tsne_vectors_filepath = os.path.join(intermediate_directory,
                                     u'tsne_vectors.npy')

In [163]:
%%time

if 0 == 0:
    
    tsne = TSNE()
    tsne_vectors = tsne.fit_transform(tsne_input.values)
    
    with open(tsne_filepath, 'wb') as f:
        pickle.dump(tsne, f)

    pd.np.save(tsne_vectors_filepath, tsne_vectors)
    
with open(tsne_filepath) as f:
    tsne = pickle.load(f)
    
tsne_vectors = pd.np.load(tsne_vectors_filepath)

tsne_vectors = pd.DataFrame(tsne_vectors,
                            index=pd.Index(tsne_input.index),
                            columns=[u'x_coord', u'y_coord'])

UnicodeDecodeError: 'charmap' codec can't decode byte 0x81 in position 34: character maps to <undefined>

Now we have a two-dimensional representation of our data! Let's take a look.

In [ ]:
tsne_vectors.head()

In [95]:
tsne_vectors[u'word'] = tsne_vectors.index

### Plotting with Bokeh

In [88]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource, value

output_notebook()

Loading BokehJS ...

In [89]:
# add our DataFrame as a ColumnDataSource for Bokeh
plot_data = ColumnDataSource(tsne_vectors)

# create the plot and configure the
# title, dimensions, and tools
tsne_plot = figure(title=u't-SNE Word Embeddings',
                   plot_width = 800,
                   plot_height = 800,
                   tools= (u'pan, wheel_zoom, box_zoom,'
                           u'box_select, resize, reset'),
                   active_scroll=u'wheel_zoom')

# add a hover tool to display words on roll-over
tsne_plot.add_tools( HoverTool(tooltips = u'@word') )

# draw the words as circles on the plot
tsne_plot.circle(u'x_coord', u'y_coord', source=plot_data,
                 color=u'blue', line_alpha=0.2, fill_alpha=0.1,
                 size=10, hover_line_color=u'black')

# configure visual elements of the plot
tsne_plot.title.text_font_size = value(u'16pt')
tsne_plot.xaxis.visible = False
tsne_plot.yaxis.visible = False
tsne_plot.grid.grid_line_color = None
tsne_plot.outline_line_color = None

# engage!
show(tsne_plot);

## Conclusion

Whew! Let's round up the major components that we've seen:
1. Text processing with **spaCy**
1. Automated **phrase modeling**
1. Topic modeling with **LDA** $\ \longrightarrow\ $ visualization with **pyLDAvis**
1. Word vector modeling with **word2vec** $\ \longrightarrow\ $ visualization with **t-SNE**

#### Why use these models?
Dense vector representations for text like LDA and word2vec can greatly improve performance for a number of common, text-heavy problems like:
- Text classification
- Search
- Recommendations
- Question answering

...and more generally are a powerful way machines can help humans make sense of what's in a giant pile of text. They're also often useful as a pre-processing step for many other downstream machine learning applications.

## Data Science @ S&P Global &mdash; *we are hiring!*